# EDA: Outlier Detection & Handling Imbalanced Data


This notebook focuses on two key data preprocessing tasks used in real-world
machine learning pipelines:

1. Outlier detection and handling (IQR, Z-score, visual inspection)
2. Handling imbalanced datasets (oversampling, SMOTE, undersampling)

These steps help improve model quality, stability, and fairness.


Install the geopy library using 'pip install geopy'. Documentation: https://pypi.org/project/geopy/

In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

You will be using a dataset that contains shipping information. Your goal is to prepare this dataset for analysis. You will be given examples of how to carry out feature engineering.

In [ ]:
#Read the csv data
df = pd.read_csv("Shipment_data.csv")
#check the shape of data
print(df.shape)
df.head()

(9222, 11)


,Order-Date,Delivery-Date,Promised-Date,Courier,Box Height,Box Width,Box Length,Box Weight,Weather,Store address,Delivery address
0,1/24/2021,1/27/2021,1/26/2021,PS,29.632656,34.722553,17.523058,3.807421,Good,'-123.048/49.1534','-126.07325247944962/51.29548801984406'
1,4/9/2021,4/13/2021,4/13/2021,FD,29.632656,34.722553,17.523058,1.902088,Good,'-123.048/49.1534','-126.07255765553835/51.29486187466757'
2,9/27/2021,10/2/2021,10/2/2021,PS,29.632656,34.722553,17.523058,2.152103,Good,'-123.048/49.1534','-126.07443435490822/51.29566033167437'
3,11/29/2020,12/4/2020,12/3/2020,PS,29.632656,34.722553,17.523058,3.802455,Good,'-123.048/49.1534','-126.07310897909636/51.295612714656855'
4,9/6/2021,9/12/2021,9/11/2021,FD,29.632656,34.722553,17.523058,5.510081,Good,'-123.048/49.1534','-126.07309709870711/51.29454616285458'


##### The data type of each column

In [ ]:
#Data type for each column
df.dtypes

,0
Order-Date,object
Delivery-Date,object
Promised-Date,object
Courier,object
Box Height,float64
Box Width,float64
Box Length,float64
Box Weight,float64
Weather,object
Store address,object


### Making Observations of the Data
- we have three date columns that are in object format and will need to be processed;
- the `courier` feature is categorical;
- three columns for the size of the box (`width`, `length`, `height`), which we can leave alone;
- the `weather` column has 2 values: `good` or `bad`;
- the `Store address` contains longitude and latitude coordinates but incorrect formatting and datatype. This will need to be split up;
- The `Delivery address` is similar to the `Store address`. This will need to be reformatted and split up into longitude and latitude.

### Feature Engineering & Selection
- engineer a new column called `'Days taken'` by calculating the difference between the `delivery date` and `order date` (this will become the target (`y`) variable for analysis);
- engineer a new column called `'Distance'` by calculating the distance;(kilometres) between the store and customer using their longitude and latitude coordinates;
- one-hot encode the `courier` and `weather` columns;
- drop the `Promised-date` column as it is not needed;

## Engineering the `'Days Taken'` column and One-hot Encoding the categorical variables.

##### [A] Convert the date columns to `datetime` format.

In [ ]:
# Make a copy of the original dataframe
df_clean = df.copy()

# Convert the date columns (Order-Date & Delivery-Date) to datetime format
df_clean['Order-Date'] = pd.to_datetime(df_clean['Order-Date'])
df_clean['Delivery-Date'] = pd.to_datetime(df_clean['Delivery-Date'])
df_clean['Promised-Date'] = pd.to_datetime(df_clean['Promised-Date'])

# Check the datatype columns
print(df_clean.dtypes)


Order-Date          datetime64[ns]
Delivery-Date       datetime64[ns]
Promised-Date       datetime64[ns]
Courier                     object
Box Height                 float64
Box Width                  float64
Box Length                 float64
Box Weight                 float64
Weather                     object
Store address               object
Delivery address            object
dtype: object


##### [A] Calculate the difference in days between the `Delivery date` and `Order date` and append this generated `'days taken'` information to the dataframe.

In [ ]:
# Create the 'Days_taken' column
df_clean['Days_taken'] = (df_clean['Delivery-Date'] - df_clean['Order-Date']).dt.days

# Display data
df_clean.head()

,Order-Date,Delivery-Date,Promised-Date,Courier,Box Height,Box Width,Box Length,Box Weight,Weather,Store address,Delivery address,Days_taken
0,2021-01-24,2021-01-27,2021-01-26,PS,29.632656,34.722553,17.523058,3.807421,Good,'-123.048/49.1534','-126.07325247944962/51.29548801984406',3
1,2021-04-09,2021-04-13,2021-04-13,FD,29.632656,34.722553,17.523058,1.902088,Good,'-123.048/49.1534','-126.07255765553835/51.29486187466757',4
2,2021-09-27,2021-10-02,2021-10-02,PS,29.632656,34.722553,17.523058,2.152103,Good,'-123.048/49.1534','-126.07443435490822/51.29566033167437',5
3,2020-11-29,2020-12-04,2020-12-03,PS,29.632656,34.722553,17.523058,3.802455,Good,'-123.048/49.1534','-126.07310897909636/51.295612714656855',5
4,2021-09-06,2021-09-12,2021-09-11,FD,29.632656,34.722553,17.523058,5.510081,Good,'-123.048/49.1534','-126.07309709870711/51.29454616285458',6


##### [A] Encode the categorical `'Courier'` column.

In [ ]:
# Use one-hot encoding to encode the 'Courier' column
df_clean = pd.get_dummies(df_clean, columns=['Courier'], prefix='Courier')

# Display data
df_clean.head()


,Order-Date,Delivery-Date,Promised-Date,Box Height,Box Width,Box Length,Box Weight,Weather,Store address,Delivery address,Days_taken,Courier_FD,Courier_PS
0,2021-01-24,2021-01-27,2021-01-26,29.632656,34.722553,17.523058,3.807421,Good,'-123.048/49.1534','-126.07325247944962/51.29548801984406',3,False,True
1,2021-04-09,2021-04-13,2021-04-13,29.632656,34.722553,17.523058,1.902088,Good,'-123.048/49.1534','-126.07255765553835/51.29486187466757',4,True,False
2,2021-09-27,2021-10-02,2021-10-02,29.632656,34.722553,17.523058,2.152103,Good,'-123.048/49.1534','-126.07443435490822/51.29566033167437',5,False,True
3,2020-11-29,2020-12-04,2020-12-03,29.632656,34.722553,17.523058,3.802455,Good,'-123.048/49.1534','-126.07310897909636/51.295612714656855',5,False,True
4,2021-09-06,2021-09-12,2021-09-11,29.632656,34.722553,17.523058,5.510081,Good,'-123.048/49.1534','-126.07309709870711/51.29454616285458',6,True,False


##### [A] Encode the categorical `'Weather'` column.

In [ ]:
# Use one-hot encoding to encode the 'Weather' column
df_clean = pd.get_dummies(df_clean, columns=['Weather'], prefix='Weather')

# Display data
df_clean.head()

,Order-Date,Delivery-Date,Promised-Date,Box Height,Box Width,Box Length,Box Weight,Store address,Delivery address,Days_taken,Courier_FD,Courier_PS,Weather_Bad,Weather_Good
0,2021-01-24,2021-01-27,2021-01-26,29.632656,34.722553,17.523058,3.807421,'-123.048/49.1534','-126.07325247944962/51.29548801984406',3,False,True,False,True
1,2021-04-09,2021-04-13,2021-04-13,29.632656,34.722553,17.523058,1.902088,'-123.048/49.1534','-126.07255765553835/51.29486187466757',4,True,False,False,True
2,2021-09-27,2021-10-02,2021-10-02,29.632656,34.722553,17.523058,2.152103,'-123.048/49.1534','-126.07443435490822/51.29566033167437',5,False,True,False,True
3,2020-11-29,2020-12-04,2020-12-03,29.632656,34.722553,17.523058,3.802455,'-123.048/49.1534','-126.07310897909636/51.295612714656855',5,False,True,False,True
4,2021-09-06,2021-09-12,2021-09-11,29.632656,34.722553,17.523058,5.510081,'-123.048/49.1534','-126.07309709870711/51.29454616285458',6,True,False,False,True


## Engineering the Distance Column


For this activity, you will be calculating the distance between two places using their longitude and latitude coordinates. There are multiple ways and libraries to carry out this calculation. We recommend using the [geopy library](https://geopy.readthedocs.io/en/stable/) for calculating the `geodesic`: https://en.wikipedia.org/wiki/Geodesic

**First, you will need to fix the formatting of the store and customer address columns. You will notice they are in text format and enclosed by quotation marks. (They are strings)**

##### [A] Remove the quotation marks in the data for in the `Store address` and `Customer address` column.
> Hint: Replace the quotation mark with an empty string.

In [ ]:
# Replace the quotation marks with empty string in the Store address and Delivery address columns
df_clean['Store address'] = df_clean['Store address'].str.replace("'", "")
df_clean['Delivery address'] = df_clean['Delivery address'].str.replace("'", "")

##### [A] Convert the `Store address` and `Delivery address` columns into numpy arrays.

In [ ]:
# Convert the Store address and Delivery address columns into numpy arrays.
store_address_array = df_clean['Store address'].values
delivery_address_array = df_clean['Delivery address'].values

print(f"Store address array shape: {store_address_array.shape}")
print(f"Delivery address array shape: {delivery_address_array.shape}")


Store address array shape: (9222,)
Delivery address array shape: (9222,)


**The longitude and lattitude cordinates are seperated in the data by a slash `'/'`**

##### [A] Loop through each element in the `'store_address_array'` and split the longitude (first value) and the latitude (second value) and append each to a new list respectively. Simultaneously, convert the longitude and latitude elements to `float` datatype.
> Hint: Use the `.split()` function on the string to split and use the `float()` function to convert to the correct data type.

In [ ]:
# Use split() method to split the longitude (first value) and the latitude (second value)
# After splitting convert the values to float
store_longitude = []
store_latitude = []

for address in store_address_array:
    lon, lat = address.split('/')
    store_longitude.append(float(lon))
    store_latitude.append(float(lat))


##### [A] Loop through each element in the `'delivery_address_array'` and split the longitude (first value) and the latitude (second value) and append each to a new list respectively. Simultaneously, convert the longitude and latitude elements to `float` datatype.

In [ ]:
# Loop through each element in the 'delivery_address_array' and split the longitude and latitude
# After splitting convert the values to float
delivery_longitude = []
delivery_latitude = []

for address in delivery_address_array:
    lon, lat = address.split('/')
    delivery_longitude.append(float(lon))
    delivery_latitude.append(float(lat))


##### [A] Now that you have 4 lists containing the longitude and latitude coordinates of both the store and customer, use the `geopy` library (via the `geodesic` method) to calculate the distance between the coordinates. Append the distance information to the dataframe.

In [ ]:
# Calculate the distance between coordinates
distance = []

for i in range(len(store_longitude)):
    store_coords = (store_latitude[i], store_longitude[i])
    delivery_coords = (delivery_latitude[i], delivery_longitude[i])

    # Calculate distance in kilometers
    dist_km = geodesic(store_coords, delivery_coords).kilometers
    distance.append(dist_km)

# Adding distance to our dataframe
df_clean['Distance_km'] = distance

df_clean.head(5)

,Order-Date,Delivery-Date,Promised-Date,Box Height,Box Width,Box Length,Box Weight,Store address,Delivery address,Days_taken,Courier_FD,Courier_PS,Weather_Bad,Weather_Good,Distance_km
0,2021-01-24,2021-01-27,2021-01-26,29.632656,34.722553,17.523058,3.807421,-123.048/49.1534,-126.07325247944962/51.29548801984406,3,False,True,False,True,321.473027
1,2021-04-09,2021-04-13,2021-04-13,29.632656,34.722553,17.523058,1.902088,-123.048/49.1534,-126.07255765553835/51.29486187466757,4,True,False,False,True,321.389102
2,2021-09-27,2021-10-02,2021-10-02,29.632656,34.722553,17.523058,2.152103,-123.048/49.1534,-126.07443435490822/51.29566033167437,5,False,True,False,True,321.543559
3,2020-11-29,2020-12-04,2020-12-03,29.632656,34.722553,17.523058,3.802455,-123.048/49.1534,-126.07310897909636/51.295612714656855,5,False,True,False,True,321.476244
4,2021-09-06,2021-09-12,2021-09-11,29.632656,34.722553,17.523058,5.510081,-123.048/49.1534,-126.07309709870711/51.29454616285458,6,True,False,False,True,321.389395


##  Feature Selection

Now that we have engineered some new columns for our dataset we can drop columns we don't need anymore.

##### [A] At your discretion, drop the columns you think we don't need anymore and keep the columns we need to proceed with further analysis and/or machine learning.

In [ ]:
# Drop the columns you think we don't need anymore
columns_to_drop = [
    'Order-Date',
    'Delivery-Date',
    'Promised-Date',
    'Store address',
    'Delivery address'
]

df_clean = df_clean.drop(columns=columns_to_drop)


In [ ]:
#Display data
df_clean.head(7)

,Box Height,Box Width,Box Length,Box Weight,Days_taken,Courier_FD,Courier_PS,Weather_Bad,Weather_Good,Distance_km
0,29.632656,34.722553,17.523058,3.807421,3,False,True,False,True,321.473027
1,29.632656,34.722553,17.523058,1.902088,4,True,False,False,True,321.389102
2,29.632656,34.722553,17.523058,2.152103,5,False,True,False,True,321.543559
3,29.632656,34.722553,17.523058,3.802455,5,False,True,False,True,321.476244
4,29.632656,34.722553,17.523058,5.510081,6,True,False,False,True,321.389395
5,29.632656,34.722553,17.523058,1.697703,5,False,True,False,True,321.607612
6,29.632656,34.722553,17.523058,2.516805,4,True,False,False,True,321.486273


In [ ]:
# Final check of the cleaned dataframe
print(f"Final dataframe shape: {df_clean.shape}")
print("\nColumns in final dataframe:")
print(df_clean.columns.tolist())
print("\nData types:")
print(df_clean.dtypes)

Final dataframe shape: (9222, 10)

Columns in final dataframe:
['Box Height', 'Box Width', 'Box Length', 'Box Weight', 'Days_taken', 'Courier_FD', 'Courier_PS', 'Weather_Bad', 'Weather_Good', 'Distance_km']

Data types:
Box Height      float64
Box Width       float64
Box Length      float64
Box Weight      float64
Days_taken        int64
Courier_FD         bool
Courier_PS         bool
Weather_Bad        bool
Weather_Good       bool
Distance_km     float64
dtype: object
